# Imports & Constants #

In [1]:
%matplotlib inline 
import os
import re
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


import keras
from keras.models import Sequential, load_model
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dropout
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.layers import LSTM
from sklearn.utils import shuffle

OUTPUT_FNAME = 'RESULT.png'

Using TensorFlow backend.


# Frame Preprocessing #
Opening Frames

In [ ]:
DATA_IN_DIR = './Data/Color/'
DATA_OUT_DIR = './Data/Grey/'

INPUT_SIZE_W = 1127
INPUT_SIZE_L = 634
OUTPUT_SIZE_W = 610
OUTPUT_SIZE_L = 610

ori_files = os.listdir(DATA_IN_DIR)

for fname in ori_files:
    img = Image.open(DATA_IN_DIR + fname).convert('L')
    img = img.crop(((INPUT_SIZE_W/2)-(OUTPUT_SIZE_W/2),(INPUT_SIZE_L/2)-(OUTPUT_SIZE_L/2), (INPUT_SIZE_W/2)+(OUTPUT_SIZE_W/2), (INPUT_SIZE_L/2)+(OUTPUT_SIZE_L/2)))
    img.save(DATA_OUT_DIR + fname)

# Halo vs NoHalo Image Prep #

In [8]:
DATA_IN_DIR = './Data/Color/'
DATA_OUT_DIR_C1 = './Data/Grey/Halo/'
DATA_OUT_DIR_C2 = './Data/Grey/NoHalo/'

INPUT_SIZE_W = 1127
INPUT_SIZE_L = 634
OUTPUT_SIZE_W = 610
OUTPUT_SIZE_L = 610

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)


for i in range(1, 2000):
    curName = 'Test1_' + str(i)
    
    # Image Result.
    img = Image.open(DATA_IN_DIR + curName + '/RESULT.png').convert('L')
    img = img.crop(((INPUT_SIZE_W/2)-(OUTPUT_SIZE_W/2),(INPUT_SIZE_L/2)-(OUTPUT_SIZE_L/2), (INPUT_SIZE_W/2)+(OUTPUT_SIZE_W/2), (INPUT_SIZE_L/2)+(OUTPUT_SIZE_L/2)))
    img.save(DATA_OUT_DIR_C1 + curName + '_R.png')
    
    ori_files = natural_sort(os.listdir(DATA_IN_DIR + curName))
    
    # Image Identification.
    img = Image.open(DATA_IN_DIR + curName + '/' + ori_files[1]).convert('L')
    img = img.crop(((INPUT_SIZE_W/2)-(OUTPUT_SIZE_W/2),(INPUT_SIZE_L/2)-(OUTPUT_SIZE_L/2), (INPUT_SIZE_W/2)+(OUTPUT_SIZE_W/2), (INPUT_SIZE_L/2)+(OUTPUT_SIZE_L/2)))
    img.save(DATA_OUT_DIR_C1 + curName + '_' + ori_files[1])
    
    # Image Tracking.
    img = Image.open(DATA_IN_DIR + curName + '/' + ori_files[30]).convert('L')
    img = img.crop(((INPUT_SIZE_W/2)-(OUTPUT_SIZE_W/2),(INPUT_SIZE_L/2)-(OUTPUT_SIZE_L/2), (INPUT_SIZE_W/2)+(OUTPUT_SIZE_W/2), (INPUT_SIZE_L/2)+(OUTPUT_SIZE_L/2)))
    img.save(DATA_OUT_DIR_C2 + curName + '_' + ori_files[30])
    
    # Image Tracking.
    img = Image.open(DATA_IN_DIR + curName + '/' + ori_files[50]).convert('L')
    img = img.crop(((INPUT_SIZE_W/2)-(OUTPUT_SIZE_W/2),(INPUT_SIZE_L/2)-(OUTPUT_SIZE_L/2), (INPUT_SIZE_W/2)+(OUTPUT_SIZE_W/2), (INPUT_SIZE_L/2)+(OUTPUT_SIZE_L/2)))
    img.save(DATA_OUT_DIR_C2 + curName + '_' + ori_files[50])
    

# Load and Split: Train & Test Data #

In [14]:
from tqdm import tqdm

DATA_DIR_C1 = './Data/Halo-NoHalo/Halo/'
DATA_DIR_C2 = './Data/Halo-NoHalo/NoHalo/'

def load_data():
    data = []
    nbImages = 0
    for fname in tqdm(os.listdir(DATA_DIR_C1)):
        label = 1
        img = Image.open(DATA_DIR_C1 + fname)
        data.append([np.array(img), np.array(label)])
        nbImages+=1

    for fname in tqdm(os.listdir(DATA_DIR_C2)):
        label = 0
        img = Image.open(DATA_DIR_C2 + fname)
        data.append([np.array(img), np.array(label)])
        nbImages+=1

    return data

data = load_data()
print(len(data))

100%|██████████| 3998/3998 [00:36<00:00, 108.91it/s]

7996


In [40]:
data = shuffle(data)
data_train = data[0:6000]
data_test = data[6000:]
print(len(data_train))
print(len(data_test))

X_train = np.array([d[0] for d in data_train])
Y_train = np.array([d[1] for d in data_train])

X_test = np.array([d[0] for d in data_train])
Y_test = np.array([d[1] for d in data_train])

print('X_train: ' + str(len(X_train)))
print('Y_train: ' + str(len(Y_train)))
print('X_test: ' + str(len(X_test)))
print('Y_test: ' + str(len(Y_test)))

X_train = X_train.reshape(X_train.shape[0], 1, 610, 610)
X_test = X_test.reshape(X_test.shape[0], 1, 610, 610)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print('Input shape: ' + str(X_train[0].shape))
print('Output shape: ' + str(Y_train[0].shape))
print('X_train shape: ' + str(X_train.shape))
print('Y_train shape: ' + str(Y_train.shape))

6000
1996
X_train: 6000
Y_train: 6000
X_test: 6000
Y_test: 6000
Input shape: (1, 610, 610)
Output shape: ()
X_train shape: (6000, 1, 610, 610)
Y_train shape: (6000,)


In [43]:
from keras.utils import np_utils

# Preprocess class labels
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

In [44]:
# 3. Import libraries and modules
import numpy as np
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

# 7. Define model architecture
model = Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1,610,610), data_format="channels_first"))
#model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
 
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
# 9. Fit model on training data
model.fit(X_train, Y_train, 
          batch_size=16, epochs=1, verbose=1)
 
# 10. Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=0)

Epoch 1/1
6000/6000 [==============================] - 12107s - loss: 8.1307 - acc: 0.4927     

# Loading Data #

In [ ]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)


DATA_TRAIN_DIR = './Data/Grey/Test_1/'

training_fnames = natural_sort(os.listdir(DATA_TRAIN_DIR))
training_fnames.remove(OUTPUT_FNAME)

output_fname = OUTPUT_FNAME

imgs = list()
for fname in training_fnames:
    imgs.append(Image.open(DATA_TRAIN_DIR + fname))

output_img = Image.open(DATA_TRAIN_DIR + output_fname)
    
print(len(imgs))

fig = plt.figure(figsize=(40,30))

for i in range(0, len(imgs)):
    f = fig.add_subplot(9,9,i+1)
    f.imshow(imgs[i],cmap='gray')

f = fig.add_subplot(9,9,len(imgs))
f.imshow(output_img, cmap='gray')
    
plt.show()

In [ ]:
for num, data in enumerate(train_data[24:36]):
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(3,4,num+1)
    orig = img_data
    data = orig #img_data.reshape(IMG_SIZE, IMG_SIZE, 3)
    
    #model_out = model.predict([data])[0]
    
    if img_num[0] == 0: str_label = 'Dog'
    else: str_label = 'Cat'
        
    y.imshow(orig)
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)


In [ ]:
# Trying this: https://arxiv.org/pdf/1504.01561.pdf

In [ ]:
# 3. Import libraries and modules
import numpy as np
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
 
# 4. Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
# 5. Preprocess input data
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
 
# 7. Define model architecture
model = Sequential()
 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1,28,28), data_format="channels_first"))
#model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
 
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
# 9. Fit model on training data
model.fit(X_train, Y_train, 
          batch_size=16, epochs=5, verbose=1)
 
# 10. Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=0)